In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import gdown
import os,glob
import pandas as pd
from keras.utils import to_categorical
from scipy import sparse
import string
import numpy as np
import json
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout,TimeDistributed,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

This code block is used to mount your Google Drive to the Colab environment. It allows you to access files stored in your Google Drive directly from the Colab notebook. The force_remount=True parameter ensures that the drive is remounted if it was previously mounted during the session. This is useful for ensuring that the latest version of the drive's contents is accessible.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


This code block imports essential libraries and modules for data processing, machine learning model building, and text analysis. It includes libraries for data manipulation (Pandas), neural network construction (Keras, TensorFlow), natural language processing (NLTK), and various utilities for handling arrays, strings, and file paths. Additionally, it sets the seed for random number generation to ensure reproducibility.

In [ ]:
import gdown
import os,glob
import pandas as pd
from keras.utils import to_categorical
from scipy import sparse
import string
import numpy as np
import json
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [ ]:
data_url = "https://drive.google.com/drive/folders/1_RGwmjvW8p3jHuKqIbbHXdBDuDl2DTjU?usp=drive_link"

In [ ]:
gdown.download_folder (data_url, quiet=True, use_cookies=False)

['/content/Guardian News Articles/guardian_articles.csv']

In [ ]:
pip install transformers


In [ ]:
data = pd.read_csv('/content/Guardian News Articles/guardian_articles.csv')

# Basic preprocessing
data['bodyContent'] = data['bodyContent'].apply(lambda x: '[CLS] ' + x + ' [SEP]')
data['webTitle'] = data['webTitle'].apply(lambda x: '[CLS] ' + x + ' [SEP]')

# Splitting the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(data['bodyContent'], data['webTitle'], test_size=0.2)


In [ ]:
# data_url = "something"

This line of code assigns a URL to the variable data_url. The URL points to a Google Drive folder containing the data files needed for the project. The data can be accessed and downloaded using this URL, typically in combination with Google Drive APIs or tools like gdown to facilitate the download process in a programming environment.

In [ ]:
data_url = "https://drive.google.com/drive/folders/1_RGwmjvW8p3jHuKqIbbHXdBDuDl2DTjU?usp=drive_link"

This code uses the gdown library to download an entire folder from Google Drive. The data_url specifies the location of the folder on Google Drive. The quiet=True parameter ensures that the download process runs without printing unnecessary logs, and use_cookies=False indicates that cookies should not be used in the download process. This is a convenient way to programmatically download all files from a shared Google Drive folder directly into the runtime environment.

In [ ]:
gdown.download_folder (data_url, quiet=True, use_cookies=False)

['/content/Guardian News Articles/cleaned_guardian_articles_test.csv',
 '/content/Guardian News Articles/cleaned_guardian_articles_train.csv',
 '/content/Guardian News Articles/cleaned_guardian_articles.csv',
 '/content/Guardian News Articles/guardian_articles.csv']

This code block is responsible for loading and cleaning a dataset. It first reads a CSV file named 'guardian_articles.csv' from the specified directory into a pandas DataFrame called data_sample. After loading the data, it removes rows where values in either the 'webTitle' or 'bodyContent' columns are missing. This cleaning step is crucial to ensure the quality and reliability of the dataset for further processing and analysis.

In [ ]:
# # Load dataset
# data = pd.read_csv('/content/Guardian News Articles/guardian_articles.csv')

# # Using only 10% of the data
# # data = data_sample.sample(frac=0.1, random_state=42)

# # Drop rows with missing values
# data.dropna(subset=['webTitle', 'bodyContent'], inplace=True)

This code block sets up the necessary components for advanced text preprocessing. It starts by downloading NLTK datasets for stopwords and WordNet, which are used for text cleaning and normalization. A function clean_text is defined to convert text to lowercase, remove punctuation and numbers, eliminate extra spaces, tokenize, remove stopwords, and apply lemmatization. This function is applied to the 'webTitle' and 'bodyContent' columns of the dataset to clean and preprocess the text. Additionally, special tokens 'starttoken' and 'endtoken' are added to each headline in 'webTitle' to signify the beginning and end, which is a common technique in natural language processing, especially in sequence modeling tasks.

In [ ]:
# Load the cleaned dataset from a CSV file
data = pd.read_csv('/content/Guardian News Articles/cleaned_guardian_articles_train.csv')

# Drop rows with missing values
data.dropna(subset=['webTitle', 'bodyContent','webTitleToken'], inplace=True)

In [ ]:
data

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id,webTitleToken
0,world/2016/jan/31/church-christian-british-adu...,World news,half british adult visited church past year,https://www.theguardian.com/world/2016/jan/31/...,half british adult visited church past year de...,2016-01-31T22:00:09Z,21.0,starttoken half british adult visited church p...
1,tv-and-radio/2016/jan/31/war-and-peace-recap-e...,Television & radio,war peace recap episode five – hero leech cast...,https://www.theguardian.com/tv-and-radio/2016/...,‘strange thing turn sometimes …’ pierre get aw...,2016-01-31T22:00:09Z,22.0,starttoken war peace recap episode five – hero...
2,sport/2016/jan/31/angelique-kerber-serena-will...,Sport,angelique kerber aim dislodge serena williams ...,https://www.theguardian.com/sport/2016/jan/31/...,gone midnight angelique kerber conducting yet ...,2016-01-31T21:59:09Z,23.0,starttoken angelique kerber aim dislodge seren...
3,football/2016/jan/31/fulham-call-off-moussa-de...,Football,fulham call moussa dembéle’s £m move tottenham...,https://www.theguardian.com/football/2016/jan/...,fulham angrily called moussa dembélé’s propose...,2016-01-31T21:56:54Z,24.0,starttoken fulham call moussa dembéle’s £m mov...
4,sport/2016/jan/31/kieran-brookes-england-squad,Sport,northampton kieran brooke drafted england squa...,https://www.theguardian.com/sport/2016/jan/31/...,england drafted northampton prop kieran brooke...,2016-01-31T21:52:14Z,25.0,starttoken northampton kieran brooke drafted e...
...,...,...,...,...,...,...,...,...
148719,world/2022/jun/21/marble-head-of-hercules-pull...,World news,marble head hercules pulled roman shipwreck si...,https://www.theguardian.com/world/2022/jun/21/...,archaeologist it’s underwater find keep giving...,2022-06-21T17:31:32Z,149835.0,starttoken marble head hercules pulled roman s...
148720,music/2022/jun/22/i-got-sick-of-talking-about-...,Music,‘i got sick talking myself’ spacey jane back m...,https://www.theguardian.com/music/2022/jun/22/...,mop curl caleb harper – spacey jane frontman g...,2022-06-21T17:30:09Z,149836.0,starttoken ‘i got sick talking myself’ spacey ...
148721,australia-news/2022/jun/22/the-small-town-with...,Australia news,small town big potato inspired global poetry win,https://www.theguardian.com/australia-news/202...,robertson small pretty town perched edge new s...,2022-06-21T17:30:09Z,149837.0,starttoken small town big potato inspired glob...
148722,australia-news/2022/jun/22/power-to-ban-citize...,Australia news,power ban citizen entering australia questione...,https://www.theguardian.com/australia-news/202...,high court decision striking home affair minis...,2022-06-21T17:30:08Z,149838.0,starttoken power ban citizen entering australi...


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))


In [ ]:
# Load pre-trained BERT
bert = TFBertModel.from_pretrained('bert-base-uncased')

# Build a custom model
input_ids = tf.keras.Input(shape=(512,), dtype='int32', name='input_ids')  # Adjust the shape as needed
attention_mask = tf.keras.Input(shape=(512,), dtype='int32', name='attention_mask')  # Adjust the shape as needed

# BERT output
bert_output = bert(input_ids, attention_mask=attention_mask)[0]  # Using the last hidden state

# BiLSTM Decoder
decoder_bilstm = Bidirectional(LSTM(256, return_sequences=True, return_state=True))
decoder_outputs, forward_h, forward_c, backward_h, backward_c = decoder_bilstm(bert_output)

# Dense layer for output
decoder_dense = Dense(num_classes, activation='softmax')  # num_classes is the size of your target vocabulary
decoder_outputs = TimeDistributed(decoder_dense)(decoder_outputs)

# Build the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)